In [1]:
!pip install pandas

In [2]:
import json
import uuid
import requests
# import the smartpynector module
import smartpynector as sp

In [3]:
# Defining constants
# Knowledge engine REST API URL
# KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
SUBSCRIBER_ID = "http://subscriber:8000/subscriber"
KNOWLEDGE_ENGINE_URL = "http://knowledge_engine:8280/rest"
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"

In [4]:
# Defining the smart connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=SUBSCRIBER_ID,
    knowledgeBaseName="Subscriber2",
    knowledgeBaseDescription="Subscriber2",
    reasonerEnabled=False,
)

In [5]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

<Response [400]>

In [6]:
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }
GRAPH_PATTERN= '''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''

In [7]:
ask_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="AskKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes=PREFIXES,
    graph_pattern=GRAPH_PATTERN,
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
).json()["knowledgeInteractionId"]
ask_ki_id

'http://subscriber:8000/subscriber/interaction/8c05a373-76f0-42d2-abd8-f00546322754'

In [8]:
binding_set = sp.perform_ask_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=SUBSCRIBER_ID,
    knowledge_interaction_id=ask_ki_id,
    ask_data=[]
).json()["bindingSet"]
binding_set

[{'room_id': '"http://0.0.0.0:8001/thermostat/rooms/1"',
  'temp': '35',
  'device_id': '"http://0.0.0.0:8001/thermostat/devices/1"',
  'meas': '<http://0.0.0.0:8001/thermostat/measurements/c9fe81b3-2009-4f91-8d3f-8235b413b7a8>',
  'timestamp': '"2023-06-18T18:29:25+00:00"'}]

In [9]:
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

sp.store_data_in_graphdb(
    graph_pattern=GRAPH_PATTERN,
    binding_set=binding_set,
    prefixes=PREFIXES,
    read_url=READ_URL,
    write_url=WRITE_URL,
)

In [10]:
react_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="ReactKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes=PREFIXES,
    argument_graph_pattern=GRAPH_PATTERN,
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
).json()["knowledgeInteractionId"]
react_ki_id

'http://subscriber:8000/subscriber/interaction/f2d90e23-9e25-4f68-8ac1-918110d966a7'

In [11]:
thermostat_actual_temperature = requests.get(
    THERMOSTAT_API_URL + "/actual_temperature"
).json()
thermostat_actual_temperature

36

In [12]:
binding_set2=sp.perform_react_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL, 
    knowledge_base_id=SUBSCRIBER_ID,              
    knowledge_interaction_id=react_ki_id, 
    argument_binding_set=[
        {
            "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
            "temp": f"{thermostat_actual_temperature}",
            "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
            "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
            "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
        }
    ],
).json()["resultBindingSet"]

2023-06-18 18:29:29 INFO getting
2023-06-18 18:29:29 INFO 200
2023-06-18 18:29:29 WARNING received unexpected status 400
2023-06-18 18:29:29 WARNING {"messageType":"error","message":"Bindings must be empty when the graph pattern is missing."}
2023-06-18 18:29:29 INFO retrying after a short timeout


getting
{"knowledgeInteractionId":"http://subscriber:8000/subscriber/interaction/99287242-7b0c-418a-8674-f7f792c3cbe8","handleRequestId":2,"bindingSet":[{"room_id":"\"http://0.0.0.0:8001/thermostat/rooms/1\"","temp":"34","device_id":"\"http://0.0.0.0:8001/thermostat/devices/1\"","meas":"<http://0.0.0.0:8001/thermostat/measurements/8d9414e9-4098-4925-b993-6cee13a3bd55>","timestamp":"\"2023-06-18T18:29:28+00:00\""}],"requestingKnowledgeBaseId":"http://0.0.0.0:8001/thermostat"}
200


2023-06-18 18:29:31 INFO getting


getting


2023-06-18 18:30:00 INFO getting



repoll
getting


2023-06-18 18:30:29 INFO getting



repoll
getting



KeyboardInterrupt

